<a href="https://colab.research.google.com/github/jaouni24/Character-Recognition-and-Subjectivity-Detection/blob/main/AI_Lab_Case_Study_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Task 1: Character Recognission

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## CNN Model

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from PIL import Image
import os

# Dataset Preparation
class CharacterDataset(Dataset):
    def __init__(self, csv_file, mapping_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.mapping = {int(line.split()[0]): chr(int(line.split()[1])) for line in open(mapping_file).readlines()}
        self.labels = self.data.iloc[:, 0].values  # First column is the label
        self.images = self.data.iloc[:, 1:].values.reshape(-1, 28, 28)  # 28x28 images
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.uint8)
        label = self.labels[idx]
        if self.transform:
            image = self.transform(Image.fromarray(image))
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the datasets
train_dataset = CharacterDataset('/content/characters.csv', '/content/mapping.txt', transform=transform)
test_dataset = CharacterDataset('/content/characters-test.csv', '/content/mapping.txt', transform=transform)

# Hyperparameters to loop over
learning_rates = [0.01, 0.001, 0.003]
batch_sizes = [32, 64]

# CNN Model Definition with configurable kernel sizes
class CNNModel(nn.Module):
    def __init__(self, num_classes, kernel_size1=5, kernel_size2=3):
        super(CNNModel, self).__init__()
        # Convolution layers with configurable kernel sizes
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=kernel_size1, stride=1, padding=kernel_size1//2),  # Kernel size 5x5 or other
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=kernel_size2, stride=1, padding=kernel_size2//2),  # Kernel size 3x3 or other
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128),  # Flattened to match the number of features after convolution
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x

# Loop over hyperparameters and train the model
for lr in learning_rates:
    for batch_size in batch_sizes:
        for kernel_size1 in [5, 3]:  # Testing different kernel sizes for the first convolutional layer
            for kernel_size2 in [3, 5]:  # Testing different kernel sizes for the second convolutional layer
                print(f"Training with Learning Rate: {lr}, Batch Size: {batch_size}, Kernel Size1: {kernel_size1}, Kernel Size2: {kernel_size2}")

                # DataLoader for current batch size
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

                # Initialize model, loss function, and optimizer for this combination
                model = CNNModel(num_classes=len(train_dataset.mapping), kernel_size1=kernel_size1, kernel_size2=kernel_size2)
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Training loop
                for epoch in range(10):
                    model.train()
                    running_loss = 0.0
                    for images, labels in train_loader:
                        images, labels = images.float(), labels  # Ensure float type for images
                        optimizer.zero_grad()
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()
                        running_loss += loss.item()
                    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

                # Testing and evaluation
                model.eval()
                y_true = []
                y_pred = []
                with torch.no_grad():
                    for images, labels in test_loader:
                        images, labels = images.float(), labels  # Ensure float type for images
                        outputs = model(images)
                        _, predicted = torch.max(outputs, 1)
                        y_true.extend(labels.numpy())
                        y_pred.extend(predicted.numpy())

                # Performance Metrics
                accuracy = accuracy_score(y_true, y_pred)
                precision = precision_score(y_true, y_pred, average='macro')
                recall = recall_score(y_true, y_pred, average='macro')
                f1 = f1_score(y_true, y_pred, average='macro')

                print(f"Results for LR: {lr}, Batch Size: {batch_size}, Kernel Size1: {kernel_size1}, Kernel Size2: {kernel_size2}")
                print(f"Accuracy: {accuracy:.4f}")
                print(f"Precision: {precision:.4f}")
                print(f"Recall: {recall:.4f}")
                print(f"F1 Score: {f1:.4f}")
                print("-" * 50)

                # Save the model for each configuration
                torch.save(model.state_dict(), f"trained_cnn_model_lr{lr}_bs{batch_size}_ks1{kernel_size1}_ks2{kernel_size2}.pth")

Training with Learning Rate: 0.01, Batch Size: 32, Kernel Size1: 5, Kernel Size2: 3


<ipython-input-2-73067a981159>:25: RuntimeWarning: invalid value encountered in cast
  image = self.images[idx].astype(np.uint8)


Epoch 1, Loss: 2.8279
Epoch 2, Loss: 1.6621
Epoch 3, Loss: 1.2974
Epoch 4, Loss: 1.1463
Epoch 5, Loss: 1.0494
Epoch 6, Loss: 0.9570
Epoch 7, Loss: 0.8682
Epoch 8, Loss: 0.8227
Epoch 9, Loss: 0.7773
Epoch 10, Loss: 0.7693
Results for LR: 0.01, Batch Size: 32, Kernel Size1: 5, Kernel Size2: 3
Accuracy: 0.6491
Precision: 0.6710
Recall: 0.6476
F1 Score: 0.6423
--------------------------------------------------
Training with Learning Rate: 0.01, Batch Size: 32, Kernel Size1: 5, Kernel Size2: 5


<ipython-input-2-73067a981159>:25: RuntimeWarning: invalid value encountered in cast
  image = self.images[idx].astype(np.uint8)


Epoch 1, Loss: 3.5311
Epoch 2, Loss: 1.6897
Epoch 3, Loss: 1.1488
Epoch 4, Loss: 0.8968
Epoch 5, Loss: 0.7294
Epoch 6, Loss: 0.6589
Epoch 7, Loss: 0.5374
Epoch 8, Loss: 0.4715
Epoch 9, Loss: 0.4103
Epoch 10, Loss: 0.3299
Results for LR: 0.01, Batch Size: 32, Kernel Size1: 5, Kernel Size2: 5
Accuracy: 0.6486
Precision: 0.6731
Recall: 0.6490
F1 Score: 0.6462
--------------------------------------------------
Training with Learning Rate: 0.01, Batch Size: 32, Kernel Size1: 3, Kernel Size2: 3


...

Epoch 1, Loss: 2.5775
Epoch 2, Loss: 1.0576
Epoch 3, Loss: 0.6823
Epoch 4, Loss: 0.5360
Epoch 5, Loss: 0.3959
Epoch 6, Loss: 0.2966
Epoch 7, Loss: 0.2692
Epoch 8, Loss: 0.2152
Epoch 9, Loss: 0.1928
Epoch 10, Loss: 0.1545
Results for LR: 0.003, Batch Size: 64, Kernel Size1: 5, Kernel Size2: 5
Accuracy: 0.7669
Precision: 0.7786
Recall: 0.7644
F1 Score: 0.7599
--------------------------------------------------
Training with Learning Rate: 0.003, Batch Size: 64, Kernel Size1: 3, Kernel Size2: 3


<ipython-input-2-73067a981159>:25: RuntimeWarning: invalid value encountered in cast
  image = self.images[idx].astype(np.uint8)


Epoch 1, Loss: 3.1261
Epoch 2, Loss: 1.4928
Epoch 3, Loss: 0.9581
Epoch 4, Loss: 0.7202
Epoch 5, Loss: 0.5822
Epoch 6, Loss: 0.4659
Epoch 7, Loss: 0.4031
Epoch 8, Loss: 0.3269
Epoch 9, Loss: 0.2820
Epoch 10, Loss: 0.2466
Results for LR: 0.003, Batch Size: 64, Kernel Size1: 3, Kernel Size2: 3
Accuracy: 0.7423
Precision: 0.7525
Recall: 0.7451
F1 Score: 0.7407
--------------------------------------------------
Training with Learning Rate: 0.003, Batch Size: 64, Kernel Size1: 3, Kernel Size2: 5


<ipython-input-2-73067a981159>:25: RuntimeWarning: invalid value encountered in cast
  image = self.images[idx].astype(np.uint8)


Epoch 1, Loss: 2.8339
Epoch 2, Loss: 1.3125
Epoch 3, Loss: 0.8662
Epoch 4, Loss: 0.6917
Epoch 5, Loss: 0.5419
Epoch 6, Loss: 0.4679
Epoch 7, Loss: 0.3502
Epoch 8, Loss: 0.3251
Epoch 9, Loss: 0.2986
Epoch 10, Loss: 0.2395
Results for LR: 0.003, Batch Size: 64, Kernel Size1: 3, Kernel Size2: 5
Accuracy: 0.7388
Precision: 0.7496
Recall: 0.7404
F1 Score: 0.7337
--------------------------------------------------


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

class CharacterDataset(Dataset):
    def __init__(self, csv_file, mapping_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.mapping = {int(line.split()[0]): chr(int(line.split()[1])) for line in open(mapping_file).readlines()}
        self.labels = self.data.iloc[:, 0].values  # First column is the label
        self.images = self.data.iloc[:, 1:].values.reshape(-1, 28, 28)  # 28x28 images
        self.transform = transform

        # Ensure labels are zero-indexed by subtracting the minimum label value
        self.min_label = min(self.labels)
        self.labels = self.labels - self.min_label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = np.uint8(image)  # Ensure the image is in uint8 format
        if self.transform:
            image = self.transform(Image.fromarray(image))
        return image, label

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale (1 channel)
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load datasets
train_dataset = CharacterDataset('/content/characters.csv', '/content/mapping.txt', transform=transform)
test_dataset = CharacterDataset('/content/characters-test.csv', '/content/mapping.txt', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Adjust the output layer to match the number of classes
num_classes = len(set(train_dataset.labels))
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

for name, param in model.named_parameters():
    if "layer4" not in name and "fc" not in name:
        param.requires_grad = False

# Training function
def train_model(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    model.to(device) # Move the model to the device
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in dataloader:
            # Ensure labels are LongTensor and on the correct device
            images = images.float().to(device)  # Move images to device and ensure float type
            labels = labels.long().to(device)  # Move labels to device and ensure long type

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

# Training
train_model(model, train_loader, criterion, optimizer)

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    report = classification_report(all_labels, all_preds, target_names=[str(c) for c in range(num_classes)])
    # print(report)

# Evaluation
evaluate_model(model, test_loader)

# Save the model
torch.save(model.state_dict(), "pretrained_emnist_model.pth")
print("Model saved as pretrained_emnist_model.pth")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-4-7891a5dcb9e5>:29: RuntimeWarning: invalid value encountered in cast
  image = np.uint8(image)  # Ensure the image is in uint8 format


Epoch 1/10, Loss: 2.6412
Epoch 2/10, Loss: 1.5127
Epoch 3/10, Loss: 1.2321
Epoch 4/10, Loss: 1.0431
Epoch 5/10, Loss: 0.9236
Epoch 6/10, Loss: 0.8109
Epoch 7/10, Loss: 0.7180
Epoch 8/10, Loss: 0.6651
Epoch 9/10, Loss: 0.5648
Epoch 10/10, Loss: 0.5183
Model saved as pretrained_emnist_model.pth


## ResNet18 Pre-Trained Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

# Define the dataset
class CharacterDataset(Dataset):
    def __init__(self, csv_file, mapping_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.mapping = {int(line.split()[0]): chr(int(line.split()[1])) for line in open(mapping_file).readlines()}
        self.labels = self.data.iloc[:, 0].values
        self.images = self.data.iloc[:, 1:].values.reshape(-1, 28, 28)
        self.transform = transform
        self.min_label = min(self.labels)
        self.labels = self.labels - self.min_label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = np.uint8(image)
        if self.transform:
            image = self.transform(Image.fromarray(image))
        return image, label

# Data transforms
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load datasets
train_dataset = CharacterDataset('/content/characters.csv', '/content/mapping.txt', transform=transform)
test_dataset = CharacterDataset('/content/characters-test.csv', '/content/mapping.txt', transform=transform)

# Hyperparameter tuning
learning_rates = [0.01, 0.001, 0.003]
batch_sizes = [32, 64]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for lr in learning_rates:
    for batch_size in batch_sizes:
        print(f"Training with Learning Rate: {lr}, Batch Size: {batch_size}")

        # Create data loaders with the current batch size
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Load and modify ResNet18
        model = models.resnet18(pretrained=True)
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_classes = len(set(train_dataset.labels))
        model.fc = nn.Linear(model.fc.in_features, num_classes)

        # Freeze layers except the last layers
        for name, param in model.named_parameters():
            if "layer4" not in name and "fc" not in name:
                param.requires_grad = False

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Move model to device
        model.to(device)

        # Training loop
        def train_model(model, dataloader, criterion, optimizer, epochs=10):
            model.train()
            for epoch in range(epochs):
                total_loss = 0
                for images, labels in dataloader:
                    images = images.float().to(device)
                    labels = labels.long().to(device)
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

        train_model(model, train_loader, criterion, optimizer)

        # Evaluation loop
        def evaluate_model(model, dataloader):
            model.eval()
            all_labels = []
            all_preds = []
            with torch.no_grad():
                for images, labels in dataloader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    _, preds = torch.max(outputs, 1)
                    all_labels.extend(labels.cpu().numpy())
                    all_preds.extend(preds.cpu().numpy())
            return all_labels, all_preds

        all_labels, all_preds = evaluate_model(model, test_loader)

        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print("=" * 50)

Training with Learning Rate: 0.01, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 1.4625
Epoch 2/10, Loss: 1.0609
Epoch 3/10, Loss: 0.9688
Epoch 4/10, Loss: 0.9078
Epoch 5/10, Loss: 0.8657
Epoch 6/10, Loss: 0.8263
Epoch 7/10, Loss: 0.8028
Epoch 8/10, Loss: 0.7758
Epoch 9/10, Loss: 0.7548
Epoch 10/10, Loss: 0.7321
Accuracy: 0.7349, Precision: 0.7404, Recall: 0.7349, F1-Score: 0.7319
Training with Learning Rate: 0.01, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 1.3638
Epoch 2/10, Loss: 0.9166
Epoch 3/10, Loss: 0.8221
Epoch 4/10, Loss: 0.7582
Epoch 5/10, Loss: 0.7096
Epoch 6/10, Loss: 0.6676
Epoch 7/10, Loss: 0.6363
Epoch 8/10, Loss: 0.5989
Epoch 9/10, Loss: 0.5710
Epoch 10/10, Loss: 0.5438
Accuracy: 0.7430, Precision: 0.7484, Recall: 0.7430, F1-Score: 0.7413
Training with Learning Rate: 0.001, Batch Size: 32


...

Epoch 1/10, Loss: 1.3078
Epoch 2/10, Loss: 0.9044
Epoch 3/10, Loss: 0.7812
Epoch 4/10, Loss: 0.6894
Epoch 5/10, Loss: 0.6246
Epoch 6/10, Loss: 0.5616
Epoch 7/10, Loss: 0.5129
Epoch 8/10, Loss: 0.4703
Epoch 9/10, Loss: 0.4319
Epoch 10/10, Loss: 0.3963
Accuracy: 0.7486, Precision: 0.7532, Recall: 0.7486, F1-Score: 0.7463
Training with Learning Rate: 0.003, Batch Size: 32


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 1.3916
Epoch 2/10, Loss: 0.9356
Epoch 3/10, Loss: 0.8229
Epoch 4/10, Loss: 0.7448
Epoch 5/10, Loss: 0.6885
Epoch 6/10, Loss: 0.6468
Epoch 7/10, Loss: 0.6004
Epoch 8/10, Loss: 0.5752
Epoch 9/10, Loss: 0.5474
Epoch 10/10, Loss: 0.5230
Accuracy: 0.7610, Precision: 0.7635, Recall: 0.7610, F1-Score: 0.7586
Training with Learning Rate: 0.003, Batch Size: 64


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 1.3235
Epoch 2/10, Loss: 0.8932
Epoch 3/10, Loss: 0.7697
Epoch 4/10, Loss: 0.6832
Epoch 5/10, Loss: 0.6110
Epoch 6/10, Loss: 0.5515
Epoch 7/10, Loss: 0.5031
Epoch 8/10, Loss: 0.4579
Epoch 9/10, Loss: 0.4214
Epoch 10/10, Loss: 0.3856
Accuracy: 0.7534, Precision: 0.7554, Recall: 0.7534, F1-Score: 0.7519


# Task 2: Subjectivity Detection

## BERT

In [ ]:
!pip install transformers torch datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load your custom dataset
train_file = "/content/train_en.tsv"
test_file = "/content/test_en_gold.tsv"

def load_data(file_path):
    dataset = load_dataset('csv', data_files=file_path, delimiter="\t", split='train')
    return dataset

train_data = load_data(train_file)
test_data = load_data(test_file)

# Map string labels to numerical labels
def label_map(label):
    return {'SUBJ': 0, 'OBJ': 1}.get(label, -1)

# Preprocess data
def preprocess_data(examples):
    examples['label'] = [label_map(label) for label in examples['label']]
    return tokenizer(examples['sentence'], padding="max_length", truncation=True, max_length=128)

train_data = train_data.map(preprocess_data, batched=True)
test_data = test_data.map(preprocess_data, batched=True)

train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Hyperparameter tuning
learning_rates = [0.01, 0.001, 2e-5]
batch_sizes = [32, 64]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Variables to track the best model
best_model = None
best_metrics = {"accuracy": 0}
best_hyperparams = {}

for lr in learning_rates:
    for batch_size in batch_sizes:
        print(f"Training with Learning Rate: {lr}, Batch Size: {batch_size}")

        # Create data loaders
        train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        test_dataloader = DataLoader(test_data, batch_size=batch_size)

        # Load and configure model
        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        model.to(device)

        # Define optimizer
        optimizer = AdamW(model.parameters(), lr=lr)

        # Training function
        def train(model, train_dataloader, optimizer, device, epochs=10):  # Reduced epochs for hyperparameter testing
            model.train()
            for epoch in range(epochs):
                total_loss = 0
                for batch in train_dataloader:
                    optimizer.zero_grad()
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['label'].to(device)
                    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    total_loss += loss.item()
                    loss.backward()
                    optimizer.step()
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_dataloader):.4f}")

        # Evaluation function
        def evaluate(model, test_dataloader, device):
            model.eval()
            predictions, true_labels = [], []
            with torch.no_grad():
                for batch in test_dataloader:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['label'].to(device)
                    outputs = model(input_ids, attention_mask=attention_mask)
                    logits = outputs.logits
                    preds = torch.argmax(logits, dim=-1)
                    predictions.extend(preds.cpu().numpy())
                    true_labels.extend(labels.cpu().numpy())
            accuracy = accuracy_score(true_labels, predictions)
            precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
            return accuracy, precision, recall, f1

        # Train and evaluate the model
        train(model, train_dataloader, optimizer, device)
        accuracy, precision, recall, f1 = evaluate(model, test_dataloader, device)
        print(f"Metrics: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

        # Update best model if current metrics are better
        if accuracy > best_metrics["accuracy"]:
            best_metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
            best_hyperparams = {"learning_rate": lr, "batch_size": batch_size}
            best_model = model.state_dict()

# Print the best model's metrics and hyperparameters
print("\nBest Model:")
print(f"Learning Rate: {best_hyperparams['learning_rate']}, Batch Size: {best_hyperparams['batch_size']}")
print(f"Metrics: Accuracy={best_metrics['accuracy']:.4f}, Precision={best_metrics['precision']:.4f}, Recall={best_metrics['recall']:.4f}, F1={best_metrics['f1']:.4f}")

Training with Learning Rate: 0.01, Batch Size: 32


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10, Loss: 1.4501
Epoch 2/10, Loss: 0.8692
Epoch 3/10, Loss: 0.7946
Epoch 4/10, Loss: 0.7707
Epoch 5/10, Loss: 0.7805
Epoch 6/10, Loss: 0.7820
Epoch 7/10, Loss: 0.8717
Epoch 8/10, Loss: 0.8379
Epoch 9/10, Loss: 0.8850
Epoch 10/10, Loss: 0.6988
Metrics: Accuracy=0.4774, Precision=0.4774, Recall=1.0000, F1=0.6462
Training with Learning Rate: 0.01, Batch Size: 64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 3.2582
Epoch 2/10, Loss: 0.9983
Epoch 3/10, Loss: 0.7802
Epoch 4/10, Loss: 0.7597
Epoch 5/10, Loss: 0.8025
Epoch 6/10, Loss: 0.6923
Epoch 7/10, Loss: 0.7317
Epoch 8/10, Loss: 0.7799
Epoch 9/10, Loss: 0.7549
Epoch 10/10, Loss: 0.8052
Metrics: Accuracy=0.4774, Precision=0.4774, Recall=1.0000, F1=0.6462
Training with Learning Rate: 0.001, Batch Size: 32


...

Epoch 1/10, Loss: 0.9251
Epoch 2/10, Loss: 0.7999
Epoch 3/10, Loss: 0.6681
Epoch 4/10, Loss: 0.6815
Epoch 5/10, Loss: 0.6704
Epoch 6/10, Loss: 0.6581
Epoch 7/10, Loss: 0.6669
Epoch 8/10, Loss: 0.6731
Epoch 9/10, Loss: 0.6619
Epoch 10/10, Loss: 0.6667
Metrics: Accuracy=0.4774, Precision=0.4774, Recall=1.0000, F1=0.6462
Training with Learning Rate: 2e-05, Batch Size: 32


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.6244
Epoch 2/10, Loss: 0.4885
Epoch 3/10, Loss: 0.3361
Epoch 4/10, Loss: 0.1777
Epoch 5/10, Loss: 0.0669
Epoch 6/10, Loss: 0.0318
Epoch 7/10, Loss: 0.0171
Epoch 8/10, Loss: 0.0169
Epoch 9/10, Loss: 0.0143
Epoch 10/10, Loss: 0.0170
Metrics: Accuracy=0.7284, Precision=0.6812, Recall=0.8103, F1=0.7402
Training with Learning Rate: 2e-05, Batch Size: 64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.6056
Epoch 2/10, Loss: 0.4542
Epoch 3/10, Loss: 0.3456
Epoch 4/10, Loss: 0.2555
Epoch 5/10, Loss: 0.1852
Epoch 6/10, Loss: 0.1364
Epoch 7/10, Loss: 0.1024
Epoch 8/10, Loss: 0.0740
Epoch 9/10, Loss: 0.0657
Epoch 10/10, Loss: 0.0516
Metrics: Accuracy=0.7037, Precision=0.6410, Recall=0.8621, F1=0.7353

Best Model:
Learning Rate: 2e-05, Batch Size: 32
Metrics: Accuracy=0.7284, Precision=0.6812, Recall=0.8103, F1=0.7402
